<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

## Train and Benchmark and Neural Decoder  with Viterbi Decoder.

This notebook contains pipeline how to train a neural decoder model for decoding convolution code over AWGN Channel at 1/2-RSC signal.

## Import required packages

In [1]:
import multiprocessing as mp

import time

import numpy as np

import commpy as cp

import tensorflow as tf

from deepcom.model import NRSCDecoder           # Neural Decoder Model

from deepcom.metrics import BER, BLER           # metrics to benchmark Neural Decoder Model

from deepcom.utils import corrupt_signal        # simulate a AWGN Channel

from deepcom.dataset import create_dataset      # Create synthetic dataset

from deepcom.dataset import data_genenerator    # data loader for Tensorflow

import  matplotlib.pyplot  as plt
%matplotlib inline

2024-12-26 23:14:36.629355: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-26 23:14:36.848061: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-12-26 23:14:37.516732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Define Hyper-paramemeters for the experiment
# Number of training data

BLOCK_LEN = 10

NUM_TRAINING_DATA = 1200

NUM_TESTING_DATA  = 1000

# Communication Algo via Deep Learning
#(page 5, last paragraph)

NOISE_TYPE ='awgn'
SNR_train = 0.0

# Network Architectures

NUM_LAYERS = 2

NUM_HIDDEN_UNITS = 40

# Hyper-parameters for training

BATCH_SIZE = 500       # depends on size of GPU, should be a factor of num_data

LEARNING_RATE = 1e-3

DROPOUT_RATE= 0.75

#
#
# CONSTRAINT_LEN = 3     # num of shifts in Conv. Encoder
# TRACE_BACK_DEPTH = 15  # (?) a parameter Viterbi Encoder

## Generate Synthetic Dataset for training/evaluation

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
from commpy.channelcoding import Trellis

#  Generator Matrix (octal representation)

G = np.array([[0o7, 0o5]]) 

M = np.array([CONSTRAINT_LEN - 1])

trellis = Trellis(M, G, feedback=0o7, code_type='rsc')

NameError: name 'CONSTRAINT_LEN' is not defined

# Create dataset 

print('Creating training data....')

# X_train shape = [NUM_TRAINING_DATA, BLOCK_LENGTH, 2]
# Y_train shape = [NUM_TRAINING_DATA, BLOCK_LENGTH, 1]

X_train, Y_train = create_dataset(
    NUM_TRAINING_DATA, 
    BLOCK_LEN, 
    trellis, 
    noise_type=NOISE_TYPE, snr=SNR_train, seed=2018)


print('Creating testing data....')
# X_test shape = [NUM_TESTING_DATA, BLOCK_LENGTH, 2]
# Y_test shape = [NUM_TESTING_DATA, BLOCK_LENGTH, 1]

X_test, Y_test = create_dataset(
    NUM_TESTING_DATA, 
    BLOCK_LEN, 
    trellis, 
    noise_type=NOISE_TYPE, snr=SNR_train, seed=1111)


print('Number of training sequences {}'.format(len(X_train)))
print('Number of testing sequences {}'.format(len(Y_test)))

# print(X_train.shape, Y_train.shape)

## Define Neural Decoder Model
# Construct Neural Decoder

In [ ]:
inputs = tf.keras.Input(shape=(None, 2))

outputs = NRSCDecoder(
    inputs, 
    is_training=True, 
    num_layers=NUM_LAYERS, 
    hidden_units=NUM_HIDDEN_UNITS, 
    dropout=DROPOUT_RATE)

model = tf.keras.Model(inputs, outputs)

# Set up training 

In [ ]:
model.compile('adam', 'binary_crossentropy', [BER])
model.summary()

## Start Training/Eval Pipeline
# Set up Data Loader using tf.Dataset

In [ ]:
train_set = data_genenerator(X_train, Y_train, BATCH_SIZE, shuffle=True)

test_set = data_genenerator(X_test, Y_test, BATCH_SIZE, shuffle=False)

# Backup best model

In [ ]:
backup = tf.keras.callbacks.ModelCheckpoint(                     
  filepath='BiGRU.keras',
  monitor='val_loss',
  save_best_only=True)

# Stop training early if the model seems to overfit

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0,
    patience=3,
    verbose=0, mode='auto')

In [ ]:
history = model.fit(
    train_set, 
    steps_per_epoch=len(X_train) // BATCH_SIZE, 
    validation_data=test_set,
    validation_steps=len(X_test) // BATCH_SIZE,
    callbacks=[early_stopping, backup],
    epochs=100)

#
#
# model = tf.keras.models.load_model('BiGRU.hdf5',{'BER': BER})

# Count of the number of epochs

In [ ]:
epochs = range(1, len(history.history['loss']) + 1)

# Visualize loss history

In [ ]:
plt.figure(figsize=(8, 6))

plt.plot(epochs, history.history['loss'], 'r--')
plt.plot(epochs, history.history['val_loss'], 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.show()

## Benchmark Neural Decoder

In [ ]:
def benchmark_neural_decoder(noisy_inputs, labels):

    # Set up data generator
    Y = np.reshape(labels, (-1, BLOCK_LEN, 1))
    X = np.reshape(np.array(noisy_inputs)[:, :2*BLOCK_LEN], (-1, BLOCK_LEN, 2))
    test_set = data_genenerator(X, Y, BATCH_SIZE, shuffle=False)

    # Make predictions in batch
    decoded_bits = model.predict(
        test_set,
        steps=len(Y) // BATCH_SIZE)

    # Compute hamming distances
    original_bits = np.reshape(Y, (-1, BLOCK_LEN)).astype(int)
    decoded_bits =  np.reshape(np.round(decoded_bits), (-1, BLOCK_LEN)).astype(int)
    hamming_dist = np.not_equal(original_bits, decoded_bits)

    return np.sum(hamming_dist, axis=1)

def benchmark_viterbi(message_bits, noisy_bits, sigma):
    
    # make fair comparison between (100, 204) convolutional code and RNN decoder
    # Reference: Author's code

    noisy_bits[-2*int(M):] = 0
    
    # Viterbi Decoder on Conv. Code
    decoded_bits = cp.channelcoding.viterbi_decode(
        coded_bits=noisy_bits.astype(float), 
        trellis=trellis,
        tb_depth=TRACE_BACK_DEPTH,
        decoding_type='unquantized')

    # Number of bit errors (hamming distance)
    hamming_dist = cp.utilities.hamming_dist(
        message_bits.astype(int),
        decoded_bits[:-int(M)])
    return hamming_dist

# #################################################################
# For every SNR_db, we generates new noisy signals
# for fair comparision.
# #################################################################

In [ ]:
def generate_noisy_input(message_bits, trellis, sigma):
    # Encode message bit
    coded_bits = cp.channelcoding.conv_encode(message_bits, trellis)
    
    # Corrupt message on BAWGN Channel
    coded_bits = corrupt_signal(coded_bits, noise_type='awgn', sigma=sigma)
    return coded_bits, message_bits

In [ ]:
viterbiBERs, viterbiBLERs = [], []
neuralBERs, neuralBLERs = [], []

In [ ]:
pool = mp.Pool(processes=mp.cpu_count())

labels = np.reshape(Y_test, (-1, BLOCK_LEN)).astype(int)

try: 
    SNRs  = np.linspace(0, 7.0, 8)
    for snr in SNRs:
        snr_linear = snr + 10 * np.log10(1./2.)
        sigma = np.sqrt(1. / (2. * 10 **(snr_linear / 10.)))
        print('[SNR]={:.2f}'.format(snr))
        
        # Generates new noisy signals
        result = pool.starmap(
            func=generate_noisy_input,  
            iterable=[(msg_bits, trellis, sigma) for msg_bits in labels])
        X, Y =  zip(*result)
        
        # #################################################################
        # BENCHMARK NEURAL DECODER 
        # #################################################################
        nn_start = time.time()
        hamm_dists = benchmark_neural_decoder(X, Y)
        nn_ber = sum(hamm_dists) / np.product(np.shape(Y))
        nn_bler = np.count_nonzero(hamm_dists) / len(Y)
        neuralBERs.append(nn_ber)
        neuralBLERs.append(nn_bler)            
        print('	Neural Decoder:  [BER]={:5.7f} [BLER]={:5.3f} -- {:3.3f}s'.format(
            nn_ber, nn_bler, time.time() - nn_start)) 
        # #################################################################
        # BENCHMARK VITERBI DECODER 
        # #################################################################
        vi_start = time.time()
        hamm_dists = pool.starmap(benchmark_viterbi, [(y, x, sigma) for x, y in zip(X, Y)])
        ber = sum(hamm_dists) / np.product(np.shape(Y))
        bler = np.count_nonzero(hamm_dists) / len(Y)
        viterbiBERs.append(ber)
        viterbiBLERs.append(bler)
        print('	Viterbi Decoder: [BER]={:5.7f} [BLER]={:5.3f} -- {:3.3f}s'.format(
              ber, bler, time.time() - vi_start))
except Exception as e:
    print(e)
finally:
    pool.close()

# Result
# ###################################
# Plot Bit Error Rate (BER) Curve
# ###################################

In [ ]:
plt.figure(figsize=(18, 7))

plt.subplot(1, 2, 1)
plt.title('Block Length = 100 || Data rate = 1/2', fontsize=14)
plt.semilogy(SNRs, neuralBERs, '-vr')
plt.semilogy(SNRs, viterbiBERs, 's--')
plt.legend(['N-RSC (SNR_train=0 dB)', 'Viterbi'], fontsize=16)
plt.xlabel('SNR', fontsize=16)
plt.xlim(xmin=SNRs[0], xmax=SNRs[-1])  # this line
plt.ylabel('BER', fontsize=16)
plt.grid(True, which='both')
plt.savefig('result_ber_block_length_1000_snr0.png')

# ###################################
# Plot Block Error Rate (BLER) Curve
# ###################################

In [ ]:
plt.subplot(1, 2, 2)
plt.title('Block Length = 100 || Data rate = 1/2', fontsize=14)
plt.semilogy(SNRs, neuralBLERs, '-vr')
plt.semilogy(SNRs, viterbiBLERs, 's--')
plt.ylabel('BLER', fontsize=16)
plt.xlabel('SNR', fontsize=16)
plt.legend(['N-RSC (SNR_train=0 dB)', 'Viterbi'], fontsize=16)
plt.xlim(xmin=SNRs[0], xmax=SNRs[-1])  # this line
plt.grid(True, which='both')
plt.savefig('result_bler_block_length_1000_snr0.png')

##  Benchmark on K = 1000

In [ ]:
_, Y_test = create_dataset(NUM_TESTING_DATA, 1000, trellis, snr=0.0, seed=1111)
viterbiBERs, viterbiBLERs = [], []
neuralBERs, neuralBLERs = []

In [ ]:
pool = mp.Pool(processes=mp.cpu_count())

labels = np.reshape(Y_test, (-1, BLOCK_LEN)).astype(int)

try: 
    SNRs  = np.linspace(0, 7.0, 8)
    for snr in SNRs:
        snr_linear = snr + 10 * np.log10(1./2.)
        sigma = np.sqrt(1. / (2. * 10 **(snr_linear / 10.)))
        print('[SNR]={:.2f}'.format(snr))
        
        # Generates new noisy signals
        result = pool.starmap(
            func=generate_noisy_input,  
            iterable=[(msg_bits, trellis, sigma) for msg_bits in labels])
        X, Y =  zip(*result)
        
        # #################################################################
        # BENCHMARK NEURAL DECODER 
        # #################################################################
        nn_start = time.time()
        hamm_dists = benchmark_neural_decoder(X, Y)
        nn_ber = sum(hamm_dists) / np.product(np.shape(Y))
        nn_bler = np.count_nonzero(hamm_dists) / len(Y)
        neuralBERs.append(nn_ber)
        neuralBLERs.append(nn_bler)            
        print('	Neural Decoder:  [BER]={:5.7f} [BLER]={:5.3f} -- {:3.3f}s'.format(
            nn_ber, nn_bler, time.time() - nn_start)) 
        # #################################################################
        # BENCHMARK VITERBI DECODER 
        # #################################################################
        vi_start = time.time()
        hamm_dists = pool.starmap(benchmark_viterbi, [(y, x, sigma) for x, y in zip(X, Y)])
        ber = sum(hamm_dists) / np.product(np.shape(Y))
        bler = np.count_nonzero(hamm_dists) / len(Y)
        viterbiBERs.append(ber)
        viterbiBLERs.append(bler)
        print('	Viterbi Decoder: [BER]={:5.7f} [BLER]={:5.3f} -- {:3.3f}s'.format(
              ber, bler, time.time() - vi_start))
except Exception as e:
    print(e)
finally:
    pool.close()

# ###################################
# Plot Bit Error Rate (BER) Curve
# ###################################

In [ ]:
plt.figure(figsize=(18, 7))

plt.subplot(1, 2, 1)
plt.title('Block Length = 100 || Data rate = 1/2', fontsize=14)
plt.semilogy(SNRs, neuralBERs, '-vr')
plt.semilogy(SNRs, viterbiBERs, 's--')
plt.legend(['N-RSC (SNR_train=0 dB)', 'Viterbi'], fontsize=16)
plt.xlabel('SNR', fontsize=16)
plt.xlim(xmin=SNRs[0], xmax=SNRs[-1])  # this line
plt.ylabel('BER', fontsize=16)
plt.grid(True, which='both')
plt.savefig('result_ber_block_length_1000_snr0.png')

# ###################################
# Plot Block Error Rate (BLER) Curve
# ###################################

In [ ]:
plt.subplot(1, 2, 2)
plt.title('Block Length = 100 || Data rate = 1/2', fontsize=14)
plt.semilogy(SNRs, neuralBLERs, '-vr')
plt.semilogy(SNRs, viterbiBLERs, 's--')
plt.ylabel('BLER', fontsize=16)
plt.xlabel('SNR', fontsize=16)
plt.legend(['N-RSC (SNR_train=0 dB)', 'Viterbi'], fontsize=16)
plt.xlim(xmin=SNRs[0], xmax=SNRs[-1])  # this line
plt.grid(True, which='both')
plt.savefig('result_bler_block_length_1000_snr0.png')